# 2. GEE – Computing city and hexagon tree coverage

In this notebook, we will use Python to query Google Earth Engine and compute the share of each city outline that is covered by trees. The same will be done for each hexagon.

In the process, we will be using the city outlines that we computed earlier and two raster datasets that display the height of the tree canopy in the entire planet, allowing us to look only at vegetation with more than 1m of height.

The [first](https://www.nature.com/articles/s41559-023-02206-6) was made by researchers from four universities: ETH Zurich, Yale University, University of Copenhagen and University of Zurich (_Lang, N. et. al._) It has a 10m resolution.

It was combined with [another one](https://sustainability.atmeta.com/blog/2024/04/22/using-artificial-intelligence-to-map-the-earths-forests/), by researchers at Meta and the World Resources Institute, with a finer 1m resolution.

We decided to combine both to reduce the incidence of false positives, which we detected on both datasets upon manual inspection of selected cities. 

The first datasetn detected trees in the waves formed on the surface of urban bodys of water, such as lagoons and havens. This happened, for example, in Lagos (Nigeria).

Meta's model often detected trees in high-density urban developments in places like São Paulo (Brazil) and New Delhi (India).

Another caveat is that the datasets don't show all the trees that exist at the same moment of time. They use a mosaic of satellite imagery taken at different times. The first dataset spans the year 2020, while the later mostly consists of images taken from 2018 to 2020.

In [1]:
# Import packages
from datetime import datetime
import ee
import geemap

In [2]:
# The main function, which submits a Google Earth Engine task.
# The task will save a CSV with the desired information on Google Drive.
def main():
    
    # Initializes an Earth Engine instance.
    # When you run this for the first time, a browser window will prompt you to login with Google
    ee.Initialize()
    
    # Initializes a map widget, so we can see if the actions make sense in the end
    Map = geemap.Map(height='540px')
    
    # A timestamp for rudimentary version control – useful when you are still
    # tweaking the function and want to know which file was outputted by the latest script.
    now = datetime.now().strftime("%d-%m-%y-%H:%M").replace(":","_")
    
    # Loads the shpafiles genereated at the previous notebook, which were
    # already uploaded to our project via the code editor
    cities = ee.FeatureCollection('projects/dw-city-tree-coverage/assets/city_outlines')
    hexagons = ee.FeatureCollection("projects/dw-city-tree-coverage/assets/city_hexagons")
    
    # Loads the aforementioned images from GEE's public library.
    tree_coverage_1m = ee.ImageCollection('projects/meta-forest-monitoring-okw37/assets/CanopyHeight').mosaic()
    tree_coverage_10m = ee.Image('users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1')
    
    # In both datasets, keep only areas with tree coverage greater than or equal to 1m
    tree_coverage_1m = tree_coverage_1m.updateMask(tree_coverage_1m.gte(1))
    tree_coverage_10m = tree_coverage_10m.updateMask(tree_coverage_10m.gte(1))
    
    # Creates a combined tree coverage image. It will keep only the tree coverage on the 1m resolution
    # dataset that matches forested areas in the 10m resolution dataset.
    tree_coverage_mask = tree_coverage_10m.gte(1)
    tree_coverage = tree_coverage_1m.updateMask(tree_coverage_mask)
    
    # Adds a population dataset, also from GHSL
    population = ee.Image('JRC/GHSL/P2023A/GHS_POP/2020')

    # Creates a Pixel Area object. This is used by Google Earth Engine
    # to quickly compute the area of raster images. We can apply masks
    # and multply them to the pixel image in order to compute total area coverage.
    pixel_area = ee.Image.pixelArea()
    
    # The function below is defined within main so it can access all the variables
    # in this scope. This is importante because it needs to access 'pixel_area', 
    # 'population' and 'tree_coverage' while being mapped using lambda. This weird construction
    # is due to my geemap inexperience. I couldn't find a more elegant way of doing it.
    def process(feature):
        
        # Computes basic polygon elements
        ft_area = feature.geometry().area()
        centroid = feature.geometry().centroid().coordinates()
        lon = centroid.get(0)
        lat = centroid.get(1)
        
        # Multiply the masked tree coverage by the area of the corresponding pixel
        # in the pixel area object.
        # Pixels with trees smaller than 1 are False (0). 0 * area == 0, so they end up ignored
        # Alternatively, pixels greater or equal to 1 are True (1). 1 * area == area, so they end up counted.
        tree_area_image = tree_coverage.gte(1).multiply(pixel_area)
        
        # Now that we have an image in which each pixel has either 0 (when there are no trees)
        # or its respective area, when there are trees, we can simpyl reduce them.
        # We will use a sum reducer over the selected feature. This means that we will sum
        # all the pixels that fall within the bounds of this particular geometry.
        tree_area = tree_area_image.reduceRegion( # Take the data on the tree_are_image computed above
            reducer = ee.Reducer.sum(), # We want to sum all the values within the geometry
            geometry = feature.geometry(), # The geometry is the feature this function is being applied to
            scale = 1, # Scale of tree coverage image (1 sqm)
            maxPixels = 1e12 # Raises an error if we get too many pixels (> 1 billion)
        ).get('cover_code') # Gets the band name 'cover_code' (which came from Meta's dataset)
        
        # We will also compute the population. This is useful for the hexagons, but will also
        # be applied to the entire city areas for convenience. The computational cost isn't
        # prohibitive, and it's better to do all the processing within a single funcion.
        pop_ft = population.reduceRegion( # Take the data on the population image (each pixel has an estimate of the people living over it)
            reducer = ee.Reducer.sum(), # We want to sum all the pixels
            geometry = feature.geometry(), # That fall within the geometry
            scale = 100, # At the scale of GHSL data (100sqm)
            maxPixels = 1e12,
        ).get('population_count') # Name of band in original data
        
        # Returning the desired elements as a new feature
        return feature.set({
            'centroid': centroid,
            'lon': lon,
            'lat': lat,
            'ft_area': ft_area,
            'tree_area': tree_area,
            'tree_pct': ee.Number(tree_area).divide(ee.Number(ft_area)), # Uses earth engine Number to divide server side
            'pop_ft': pop_ft
        })
    
    # Applies the function to the cities
    city_results = cities.map(lambda feature: process(feature))

    # And then to the hexagons
    hexagon_results = hexagons.map(lambda feature: process(feature))
    
    # Export the results as a CSV file
    geemap.ee_export_vector_to_drive(
        collection = city_results,
        folder =  'DWTreeCoverAnalysis',
        description = f'TreeCoverAnalysis-Cities-{now}',
        fileFormat = 'CSV',
        selectors = ['ID_HDC_G0',
                     'lon', 'lat', 
                     'tree_area', 'ft_area', 'tree_pct', 'pop_ft']
    )
    
    geemap.ee_export_vector_to_drive(
        collection = hexagon_results,
        folder =  'DWTreeCoverAnalysis',
        description = f'TreeCoverAnalysis-Hexagons-{now}',
        fileFormat = 'CSV',
        selectors = ['city_id', 'hexagon_n',
                     'lon', 'lat', 
                     'tree_area', 'ft_area', 'tree_pct', 'pop_ft']
    )
 
    # Adds each of the images to the Map, so we can see if its behaving like expected.
    Map.addLayer(cities, {'color': 'gray'}, 'City Polygons')
    Map.addLayer(hexagons, {'color': 'red'}, 'Hexagons')
    Map.addLayer(tree_coverage_1m, {'palette': '#AAFF00'}, 'Meta – 1m tree coverage (>= 1m height)')
    Map.addLayer(tree_coverage_10m, {'palette': '#088F8F'}, 'Lang, N. et. al. – 10m tree coverage (>= 1m height)')
    Map.addLayer(tree_coverage, {'palette': '#454B1B'}, 'Combined – tree coverage (>= 1m height)')
    Map.addLayer(population, {'palette': ['white', 'purple']}, 'Population')

    
    return Map

In [3]:
if __name__ == "__main__":
    Map = main()

Exporting TreeCoverAnalysis-Cities-27-08-24-08_32... Please check the Task Manager from the JavaScript Code Editor.
Exporting TreeCoverAnalysis-Hexagons-27-08-24-08_32... Please check the Task Manager from the JavaScript Code Editor.


In [4]:
Map

No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm: 148116daaa364b72b97abc32156e501f
No such comm:

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

No such comm: 846f3d02db524c3dafa2456eba7a33fe
No such comm: cf703d6cf9a647ccb4f10848016cd884
No such comm: 8a2d7b1d095b4b46b96d31622037f182
No such comm: b13c3d2e5fa84bfaa044b9a75420d859
No such comm: f18610f3ec154f3b9c631633857221eb
No such comm: 4963b60d90a3481ab8ccad72d1757ba9
No such comm: 32e748b187ef43a08723132f29702889
